In [17]:
from sentence_transformers import SentenceTransformer
import requests
import json
from elasticsearch import Elasticsearch
import numpy as np
from tqdm.auto import tqdm

In [6]:
model = SentenceTransformer('multi-qa-distilbert-cos-v1')

c:\Users\HSUPC\python_files\新增資料夾\.venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HSUPC\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\HSUPC\python_files\新增資料夾\.venv\lib\site-packages\transformers\token

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [22]:
len(documents)

948

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

## Q1

##### Getting the embeddings model
##### First, we will get the embeddings model multi-qa-distilbert-cos-v1 from the Sentence Transformer library

##### from sentence_transformers import SentenceTransformer
##### embedding_model = SentenceTransformer(model_name)
##### Create the embedding for this user question:

##### user_question = "I just discovered the course. Can I still join it?"
##### What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

In [128]:
question = 'I just discovered the course. Can I still join it'
q = model.encode(question)
q[0]

0.07281464

## Q2. Creating the embeddings
##### Now for each document, we will create an embedding for both question and answer fields.

##### We want to put all of them into a single matrix X:

##### Create a list embeddings
##### Iterate over each document
##### qa_text = f'{question} {text}'
##### compute the embedding for qa_text, append to embeddings
##### At the end, let X = np.array(embeddings) (import numpy as np)
##### What's the shape of X? (X.shape). Include the parantheses.

In [130]:
documents = [d for d in documents if d['course']=='machine-learning-zoomcamp']

### We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [131]:
embeddings = []

for d in tqdm(documents):
    if d['course'] != 'machine-learning-zoomcamp':
        continue
    qa_text = f"{d['question']}{d['text']}"
    v_qa_text = model.encode(qa_text)
    embeddings.append(v_qa_text)
x = np.array(embeddings)
x.shape


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:59<00:00,  6.29it/s]


(375, 768)

In [132]:
qa_text

'Any advice for adding the Machine Learning Zoomcamp experience to your LinkedIn profile?I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan'

## Q3. Search
##### We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

##### The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the cosine similarity, it's sufficient to multiply the matrix X by the vector v:

##### scores = X.dot(v)
##### What's the highest score in the results?

* 65.0
* 6.5
* 0.65
* 0.065

In [133]:
scores = x.dot(q)

In [134]:
max(scores)

0.65601325

## Vector search
##### We can now compute the similarity between a query vector and all the embeddings.

##### Let's use this to implement our own vector search

'''

    class VectorSearchEngine():
        def __init__(self, documents, embeddings):
            self.documents = documents
            self.embeddings = embeddings
    
        def search(self, v_query, num_results=10):
            scores = self.embeddings.dot(v_query)
            idx = np.argsort(-scores)[:num_results]
            return [self.documents[i] for i in idx]
    
    search_engine = VectorSearchEngine(documents=documents, embeddings=X)
    search_engine.search(v, num_results=5)

'''
##### If you don't understand how the search function work:

##### Ask ChatGTP or any other LLM of your choice to explain the code
##### Check our pre-course workshop about implementing a search engine here
##### (Note: you can replace argsort with argpartition to make it a lot faster)

In [36]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [135]:
search_engine = VectorSearchEngine(documents=documents, embeddings=x)
search_engine.search(q, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine
##### Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

##### First, load the ground truth dataset:



'''

    import pandas as pd
    
    base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
    relative_url = '03-vector-search/eval/ground-truth-data.csv'
    ground_truth_url = f'{base_url}/{relative_url}?raw=1'
    
    df_ground_truth = pd.read_csv(ground_truth_url)
    df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
    ground_truth = df_ground_truth.to_dict(orient='records')
    
'''
##### Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

##### What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [39]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [92]:
ground_truth[0:2]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [140]:
response = search_engine.search(model.encode(ground_truth[0]['question']), num_results= 5)
k = list(a['id'] for a in response)
k

['0a278fb2', '39fda9f0', 'ee58a693', '4930aa19', '67e2fd13']

In [141]:
result = []

for i in tqdm(ground_truth):
    vector_q = model.encode(i['question'])
    response = search_engine.search(vector_q, num_results= 5)
    relevance = [i['document']==d['id'] for d in response]
    result.append(relevance)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:14<00:00, 24.45it/s]


In [142]:
def hit(relevance_total):
    cnt = 0
    for i in relevance_total:
        if True in i:
            cnt +=1
    return cnt/len(relevance_total)

In [145]:
hit(result)

0.9371584699453552

## Q5. Indexing with Elasticsearch
##### Now let's index these documents with elasticsearch

##### Create the index with the same settings as in the module (but change the dimensions)
##### Index the embeddings (note: you've already computed them)
##### After indexing, let's perform the search of the same query from Q1.

##### What's the ID of the document with the highest score?

In [180]:
for i in tqdm(range(len(documents))):
    documents[i]['embeddings'] = embeddings[i]
documents[0]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:00<00:00, 93767.97it/s]


{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'embeddings': array([ 8.80590528e-02,  1.55935511e-02,  7.92557672e-02,  2.52758078e-02,
         7.55765215e-02, -3.90597060e-02, -4.13814262e-02,  2.52918135e-02,
         2.43241396e-02,  3.62583739e-03, -7.28287455e-03, -3.28751579e-02,
         6.12956397e-02, -5.71100935e-02,  1.16774580e-02, -1.79442074e-02,
         4.49206308e-02, -5.41606247e-02, -1.92259368e-03,  1.48329744e-02,
         7.91359879e-03, -3.43127288e-02,  1.2148

In [147]:
es_client3 = Elasticsearch('http://localhost:9200')

In [149]:
es_client3.info()

ObjectApiResponse({'name': 'c94a9fa56e92', 'cluster_name': 'docker-cluster', 'cluster_uuid': '87OVNLdITq6vU8NS6bsMBw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [154]:
index_name = 'homework3'

In [185]:
index_setting = {
    'settings': {
        'number_of_shards':1,
        'number_of_replicas': 0
    },
    'mappings':{
        'properties':{
            'text': {'type':'text'},
            'section': {'type':'text'},
            'question': {'type':'text'},
            'course': {'type':'keyword'},
            'id': {'type':'keyword'},
            'embeddings': {
                'type':'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            }
        }
    }
}

In [247]:
es_client3.indices.delete(index= index_name, ignore_unavailable= True)
es_client3.indices.create(index= index_name, body= index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework3'})

In [215]:
documents[0:2]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872',
  'embeddings': array([ 8.80590528e-02,  1.55935511e-02,  7.92557672e-02,  2.52758078e-02,
          7.55765215e-02, -3.90597060e-02, -4.13814262e-02,  2.52918135e-02,
          2.43241396e-02,  3.62583739e-03, -7.28287455e-03, -3.28751579e-02,
          6.12956397e-02, -5.71100935e-02,  1.16774580e-02, -1.79442074e-02,
          4.49206308e-02, -5.41606247e-02, -1.92259368e-03,  1.48329744e-02,
          7.91359879e-03, -3.43127288e-

In [248]:
for doc in documents:
    es_client3.index(index = index_name, body = doc)

In [217]:
es_query = {
    'field': 'embeddings',
    'query_vector' : q,
    'k' :5,
    'num_candidates': 10000
}

In [218]:
responseq = es_client3.search(index= index_name, knn =es_query, source=['question', 'text', 'id'])

In [222]:
responseq['hits']['hits'][0]

{'_index': 'homework3',
 '_id': 'dZFLeZEBuMBkxYRmmQe7',
 '_score': 0.8280066,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

In [220]:
responseq['hits']['hits'][0]['_source']['id']

'ee58a693'

## Q6. Hit-rate for Elasticsearch
##### The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

##### Elasticsearch uses approximate techniques to make it faster.

##### Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

##### What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

In [211]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [213]:
def esquery(q):
    es_query = {
        'field': 'embeddings',
        'query_vector' : model.encode(q),
        'k' :5,
        'num_candidates': 10000
    }
    return es_query

In [249]:
testel = es_client3.search(index= index_name, knn = esquery(ground_truth[0]['question']), source = ['id'])
testel['hits']['hits']

[{'_index': 'homework3',
  '_id': 'Z5FPe5EBuMBkxYRmngp8',
  '_score': 0.74546313,
  '_source': {'id': '0a278fb2'}},
 {'_index': 'homework3',
  '_id': 'V5FPe5EBuMBkxYRmmgqf',
  '_score': 0.74376386,
  '_source': {'id': '39fda9f0'}},
 {'_index': 'homework3',
  '_id': 'ZJFPe5EBuMBkxYRmnQrI',
  '_score': 0.7434543,
  '_source': {'id': 'ee58a693'}},
 {'_index': 'homework3',
  '_id': 'Y5FPe5EBuMBkxYRmnQqM',
  '_score': 0.7397144,
  '_source': {'id': '4930aa19'}},
 {'_index': 'homework3',
  '_id': 'XJFPe5EBuMBkxYRmmwrL',
  '_score': 0.7251109,
  '_source': {'id': '67e2fd13'}}]

In [253]:
el_result = []

for i in tqdm(ground_truth):
    el_response = es_client3.search(index= index_name, knn = esquery(i['question']), source = ['id'])
    el_response2 = el_response['hits']['hits']
    el_result.append([i['document']==d['_source']['id'] for d in el_response2])
    

hit(el_result)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [03:00<00:00, 10.14it/s]


0.9371584699453552